In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import dask.dataframe as dd
import os
import subprocess
import gzip


In [14]:
def process_date(directory_name):
    accumulator_df = None

    for filename in glob.glob(f"../{directory_name}/18.csv.gz"):
        df = pd.read_csv(filename, sep='\t', compression='gzip')
        if accumulator_df is None:
            accumulator_df = df
        else:
            accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
        del df # release memory

    # Now you can use accumulator_df
    return accumulator_df


In [19]:
mempool_day = process_date("data/20210102")
mempool_day

# asdf = test.head(10)
# asdf

C:\Users\Deniz\AppData\Local\Temp\ipykernel_14232\852406204.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep='\t', compression='gzip')


,detecttime,hash,status,region,reorg,replace,curblocknumber,failurereason,blockspending,timepending,...,network,type,maxpriorityfeepergas,maxfeepergas,basefeepergas,dropreason,rejectionreason,stuck,gasused,detect_date
0,2021-01-02T18:00:00.000Z,0x641201377acbfa4f9ff1dad523a1feb0e407252e82f1...,pending,us-east-1,NaN,NaN,11576486,NaN,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-02
1,2021-01-02T18:00:00.004Z,0xa42a76e1650432888442fe74ec98272a0a1f254859d1...,rejected,eu-central-1,NaN,NaN,11576486,none,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,nonce too low,NaN,NaN,2021-01-02
2,2021-01-02T18:00:00.004Z,0x9aaed4cb87281406f4f71846d2e9e800d41fe9ba310d...,rejected,eu-central-1,NaN,NaN,11576486,none,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,nonce too low,NaN,NaN,2021-01-02
3,2021-01-02T18:00:00.004Z,0xace38dc0fd1598cf945088a07e3968c373290e05e19c...,rejected,eu-central-1,NaN,NaN,11576486,none,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,nonce too low,NaN,NaN,2021-01-02
4,2021-01-02T18:00:00.004Z,0xaa0278de0014fd83033e994b234000f79977f2557b96...,rejected,eu-central-1,NaN,NaN,11576486,none,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,nonce too low,NaN,NaN,2021-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589202,2021-01-02T18:59:59.869Z,0xbe197adacba4b3925520ac1b023ee35350bab9eb4235...,pending,us-east-1,NaN,NaN,11576784,NaN,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-02
589203,2021-01-02T18:59:59.919Z,0x41e42a345ecfa9bd3065220f05e20f7d321f74ac3c2e...,pending,us-east-1,NaN,NaN,11576784,NaN,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-02
589204,2021-01-02T18:59:59.922Z,0xfc2f2bf914753e6b7aa91e03dd569e422093219b54c1...,rejected,eu-central-1,NaN,NaN,11576784,none,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,nonce too low,NaN,NaN,2021-01-02
589205,2021-01-02T18:59:59.941Z,0xbe197adacba4b3925520ac1b023ee35350bab9eb4235...,pending,ap-southeast-1,NaN,NaN,11576784,NaN,NaN,NaN,...,main,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-02


In [22]:
# Apply conditions to filter the DataFrame
filtered_df = mempool_day.loc[
    (mempool_day['status'] == 'confirmed') & 
    (mempool_day['network'] == 'main') & 
    (mempool_day['region'] == 'us-east-1')
]

# columns_to_drop = ['reorg', 'replace', 'gas', 'value', 'toaddress', 'fromaddress', 'input', 'type', 'maxpriorityfeepergas', 'maxfeepergas', 'basefeepergas', 'dropreason', 'rejectionreason', 'stuck', 'status', 'region', 'failurereason', 'network']

# Drop the columns from the DataFrame
# filtered_df = filtered_df.drop(columns=columns_to_drop, axis=1)

# Count the total number of rows in the filtered DataFrame
total_rows_filtered = filtered_df.shape[0]

# Count the number of unique values in the 'hash' column of the filtered DataFrame
unique_hashes_filtered = filtered_df['hash'].nunique()

# Check if the number of unique hashes in the filtered DataFrame matches its total number of rows
if total_rows_filtered == unique_hashes_filtered:
    print("Every row in the filtered DataFrame has a unique 'hash' value.")
else:
    print(f"In the filtered DataFrame, there are {total_rows_filtered - unique_hashes_filtered} rows with non-unique 'hash' values.")

nonunique_hashes = filtered_df[filtered_df['hash'] == '0xc1228d7afe8f601fd9181f23c7f6ba8402cde62f04911b3fc7406b05d901be58']


In the filtered DataFrame, there are 1 rows with non-unique 'hash' values.


In [ ]:
if 'dropreason' in nonunique_hashes:
    nonunique_hashes = nonunique_hashes.drop(columns=['dropreason'])

In [ ]:
expected_num_fields = 27  # Adjust this based on the expected number of columns
file_path = 'data/20210423/18.csv.gz'  # Update this to the path of your gzip file
bad_lines_count = 0  # Initialize a counter for lines with an unexpected number of fields

with gzip.open(file_path, 'rt') as file:  # 'rt' mode for reading as text
    for i, line in enumerate(file):
        num_fields = len(line.split('\t'))  # Assuming a tab-separated file
        if num_fields != expected_num_fields:
            print(f"Line {i+1} has {num_fields} fields: {line}")
            bad_lines_count += 1  # Increment the counter for each bad line

print(f"Total number of lines with an unexpected number of fields: {bad_lines_count}")


In [ ]:
private_transactions = mempool_day[mempool_day["timepending"] == 0]
print(private_transactions)

In [ ]:
#mempool = pd.read_csv('15.csv.gz', sep='\t', compression='gzip')
#mempool.columns = mempool.columns.str.strip()
#mempool.columns

target_directory = "20210602"
accumulator_df = None

for filename in glob.glob(f"{target_directory}/*.csv.gz"):
  df = pd.read_csv(filename, sep='\t', compression='gzip')
  if accumulator_df is None:
    accumulator_df = df
  else:
    accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
  del df # release memory

# Now you can use accumulator_df
mempool = accumulator_df

In [ ]:
private_transactions = mempool[mempool["timepending"] == 0]

# Print the selected DataFrame
print(private_transactions)

In [ ]:
%%bash
./download_slices.sh {20210602}

In [ ]:
def run_download_script(arguments):
  # Construct the command
  command = ["/bin/bash", "./download_slices.sh"] + arguments

  # Run the command with subprocess
  result = subprocess.run(command, capture_output=True)

  # Handle success or failure based on return code and output
  if result.returncode == 0:
    print("Download script completed successfully!")
  else:
    print("Error running download script:")
    print(result.stderr.decode())

In [ ]:
# Function to download and combine data
def download_and_combine_data(date):
  """Downloads data for the specified date, extracts, and combines into a DataFrame.

  Args:
    date: The date in YYYYMMDD format (e.g., "20240215").

  Returns:
    A pandas DataFrame containing the combined data.
  """

  # Create directory for downloaded files
  data_dir = f"{date}"
  os.makedirs(data_dir, exist_ok=True)  # Create directory safely

  run_download_script([date])


  # Check if downloaded files exist
  if not os.path.exists(data_dir):
    print(f"Data directory not found for date: {date}")
    return None

  accumulator_df = None
  # Combine all CSV.gz files into a single DataFrame
  for filename in glob.glob(os.path.join(data_dir, "*.csv.gz")):
    df = pd.read_csv(filename, sep='\t', compression='gzip')
    if accumulator_df is None:
      accumulator_df = df
    else:
      accumulator_df = pd.concat([accumulator_df, df], ignore_index=True)
    del df # release memory

  mempool = accumulator_df

  # Clean up extracted files (optional)
  # for filename in glob.glob(os.path.join(data_dir, "*.csv")):
  #   os.remove(filename)

  return mempool


In [ ]:
# Example usage
date = "20210602"
mempool = download_and_combine_data(date)

if mempool is not None:
  print("Successfully combined data into a DataFrame:")
  display(mempool.head())
else:
  print("Data download or processing failed.")

# private_transactions = mempool[mempool["timepending"] == 0]
# # Print the selected DataFrame
# print(private_transactions)
# print("daamn")

In [ ]:
mempool